<img style="float: left;;" src='Figures/iteso.jpg' width="100" height="200"/></a>

# <center> <font color= #000047> Intervalos </font> </center>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.linear_model import LinearRegression

In [ ]:
data=pd.read_csv('h2o.csv')
data.head()

In [ ]:
data.tail()

In [ ]:
data['fecha'].dtype

In [ ]:
data['fecha']=pd.to_datetime(data['fecha'])
data['año']=data.fecha.dt.year
data['mes']=data.fecha.dt.month
data.head()

In [ ]:
datetime.datetime.timestamp(data['fecha'][0])

In [ ]:
data['seg']=data['fecha'].map(datetime.datetime.timestamp)
w=data['seg']/(60*60*24*365.2425)*2*np.pi
data['sin t']=np.sin(w)
data['cos t']=np.cos(w)
data[['sin t','cos t']].plot(grid=True,figsize=(15,5))

In [ ]:
data['x'].plot(grid=True,figsize=(15,5))

# Extracción de características por intervalos

In [ ]:
data.head(2)

In [ ]:
me=[]
std=[]
tendencia=[]
w=6
x=data['x']
lin=LinearRegression()
for i in range(len(x)-w):
    me.append(x[i:i+w].mean())
    std.append(x[i:i+w].std())
    t=data['seg'][i:i+w].values/(60*60*24*365.2425)
    lin.fit(t.reshape(-1,1),x[i:i+w])
    tendencia.append(lin.coef_[0])

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(x)
plt.plot(data.index[w:],me,label='media')
plt.legend()
plt.grid()

plt.figure(figsize=(15,5))
plt.plot(std,label='std')
plt.plot(tendencia,label='tendencia')
plt.legend()
plt.grid()

# Regresión

In [ ]:
from scipy.stats import zscore

In [ ]:
X=zscore(np.array([me,std,tendencia,data['sin t'].values[w:],data['cos t'].values[w:]]).T)
Y=zscore(data['x'][w:])
X.shape,Y.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
nnet=keras.Sequential([
                       keras.Input(shape=(5,)),
                       layers.Dense(5,activation=tf.nn.tanh),
                       layers.Dense(1,activation=tf.nn.tanh)
])
nnet.compile(loss='mean_squared_error',optimizer='sgd')

In [ ]:
nnet.summary()

In [ ]:
nnet.fit(X,Y/2,epochs=500)

In [ ]:
y=nnet(X)
plt.plot(y,label='regresión')
plt.plot(Y/2,label='Serie')
plt.legend()
plt.grid()